# Postprocessing of the data of bulk Aluminium

This notebook will serve as a template to post-process the different inputs of the FT of $\chi^0(\boldsymbol{q}, \boldsymbol{G}, \boldsymbol{G'})$. Different test will be experimented to test convergence and accuracy of the results. This notebook deals in parallel with primitive and conventional cells.

Visualization tools will also be available to go further in the analysis.



In [1]:
import numpy as np
import math 
import cmath
import abipy
from abipy.electrons.scr import ScrFile
import time
import matplotlib.pyplot as plt
import pointcloud as pc
from scipy.interpolate import RegularGridInterpolator
import plotly.graph_objects as go
%load_ext autoreload
%autoreload 2
import Fourier_tool as Ft
import XGChi0 
import MatrixCharacteristic
from abipy import abilab
import abipy.data as abidata
from abipy.abilab import Structure
import random


chi0_symb = '\\chi^0(\\boldsymbol{q}, \\boldsymbol{G}_1, \\boldsymbol{G}_2)'

## List of Abinit files (pay attention to always specify ngkpt and nsym with the filename): 

- Al_Bulk_eps17_SUS.nc (generic file : nband = 150, ecuteps = 17, ngkpt = [4, 4, 4], nsym = 0)

- Al_Bulk_nkpt4_nbando_DS1_SUS.nc, Al_Bulk_nkpt4_nbando_DS2_SUS.nc, Al_Bulk_nkpt4_nbando_DS3_SUS.nc, Al_Bulk_nkpt4_nbando_DS4_SUS.nc, Al_Bulk_nkpt4_nbando_DS5_SUS.nc (by increasing number of bands [150 -> 300] with ecuteps = 9, ngkpt = [4, 4, 4], nsym = 0)

- Al_Bulk_nkpt4o_DS1_SUS.nc, Al_Bulk_nkpt4o_DS2_SUS.nc, Al_Bulk_nkpt4o_DS3_SUS.nc, Al_Bulk_nkpt4o_DS4_SUS.nc, Al_Bulk_nkpt4o_DS5_SUS.nc, Al_Bulk_nkpt4o_DS6_SUS.nc (by increasing ecuteps [5 - 25] with nband = 150, ngkpt = [4, 4, 4], nsym = 0)

- Al_Bulko_DS1_SUS.nc (Conv cell, ngkpt = [4, 4, 4], nsym = 0)

- Al_Bulk_nkpt6o_DS1_SUS.nc (ngkpt = [6, 6, 6], ecuteps < 6, nsym = 0)

- Al_Bulk_nkpt8o_DS1_SUS.nc, Al_Bulk_nkpt8o_DS2_SUS.nc (ngkpt = [8, 8, 8], ecuteps = 3.6, 17, nsym = 0)

- Al_Bulk_nkpt10o_DS1_SUS.nc (ngkpt = [10, 10, 10], ecuteps = 17, nsym = 0)

- Al_Bulk_nsym0o_DS1_SUS.nc, Al_Bulk_nsym1o_DS1_SUS.nc (ngkpt = [4, 4, 4], ecuteps = 3.6, nsym =[0,1])

## Choose the file you want to analyse

Don't forget to specify the ngkpt and nsym variables from the data in the list above

In [23]:
filename = "Al_Bulk_nkpt4o_DS5_SUS.nc"
ngkpt = [4, 4, 4] #has to be entered manually from the data given above
nsym_var = 0

## Get all the information on the structure and visualize the supercell

In [3]:
structure=abipy.core.structure.Structure.from_file(filename)
structure

Structure Summary
Lattice
    abc : 2.8535866547158504 2.8535866547158504 2.8535866547158504
 angles : 59.999999999999986 59.99999999999999 59.99999999999999
 volume : 16.430780655166938
      A : 1.6475190232561199 0.0 2.3299437469564794
      B : -0.8237595116280599 1.426793327357925 2.3299437469564794
      C : -0.8237595116280599 -1.426793327357925 2.3299437469564794
PeriodicSite: Al (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]

In [88]:
structure.visualize("vesta")

Writing data to: .xsf with fmt: xsf
Executing MacOSx open command: open -a vesta --args   /Users/acloots/Documents/GitHub/Thesis_SURFASCOPE/.xsf


0

In [89]:
dfs = abilab.dataframes_from_structures([structure], index=[".nc"])
dfs.lattice

,formula,natom,alpha,beta,gamma,a,b,c,volume,abispg_num,spglib_symb,spglib_num,spglib_lattice_type
.nc,Al4,4,90.0,90.0,90.0,4.03893,4.03893,4.03893,65.886886,0,Fm-3m,225,cubic


In [60]:
dict_struct = structure.to_abivars()
natom = dict_struct["natom"]
xred = dict_struct["xred"]

In [61]:
lattice = structure.lattice.matrix
A, B, C = lattice[0], lattice[1], lattice[2]

In [70]:
XGChi0.Supercell_Vis(A, B, C, ngkpt, natom, xred)

## Lists of the generic information of the $\chi^0(\boldsymbol{q}, \boldsymbol{G}, \boldsymbol{G'})$ given by abinit

In [24]:
sus_ncfile = ScrFile(filename)
nw=sus_ncfile.reader.nw
nwr = sus_ncfile.reader.nrew
nwi = sus_ncfile.reader.nimw
ng=sus_ncfile.reader.ng 
kpoints=sus_ncfile.reader.kpoints
nkpt = len(kpoints)
chi0=sus_ncfile.reader.read_wggmat(kpoints[0])
chi0_mat = chi0.wggmat[0]
nsym = len(abipy.core.symmetries.AbinitSpaceGroup.from_structure(structure).symrec)
print("Data for filename\nNumber of frequencies = ",nw, "(",nwr, "real frequencies and ", nwi, "imaginary frequencies)\nNumber of K-points = ", nkpt,"\nNumber of G-vector in the G-sphere = ", ng,"\n$%s$" %chi0_symb,'=',chi0_mat.shape, "\nThe number of symmetry to be used to rebuild the BZ = ", nsym)

Data for filename
Number of frequencies =  14 ( 10 real frequencies and  4 imaginary frequencies)
Number of K-points =  8 
Number of G-vector in the G-sphere =  531 
$\chi^0(\boldsymbol{q}, \boldsymbol{G}_1, \boldsymbol{G}_2)$ = (531, 531) 
The number of symmetry to be used to rebuild the BZ =  48


In [25]:
G1=chi0.gsphere.gvecs

In [26]:
print(G1)

[[ 0  0  0]
 [ 1  0  0]
 [-1  0  0]
 ...
 [ 1  5  3]
 [ 2  5  4]
 [-2 -5 -4]]


In [27]:
print(G)

[[ 0  0  0]
 [ 1  0  0]
 [-1  0  0]
 ...
 [-4  0 -4]
 [ 0 -4 -4]
 [ 0  4  4]]


In [28]:
for i in range(len(G1)):
    print(G1[i]-G[i])

[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]


In [66]:
chi0 = sus_ncfile.reader.read_wggmat(kpoints[4])
chi0_mat = chi0.wggmat[omega]
print(chi0_mat)

[[-9.70474109e-02-1.17336849e-10j  4.85002101e-06+7.69820414e-04j
  -1.76893936e-05-5.03836898e-04j ...  7.38730421e-07-8.33450088e-07j
   1.06983116e-07-2.13041830e-07j  3.39570619e-07-8.75095566e-07j]
 [ 4.85002101e-06-7.69820414e-04j -8.43907744e-02+9.50686058e-11j
   2.47466844e-03-1.09511802e-06j ...  5.88204728e-07-1.28884835e-06j
  -3.14634683e-08-9.98407927e-07j -3.77717527e-07+3.87272792e-07j]
 [-1.76893936e-05+5.03836898e-04j  2.47466844e-03+1.09511802e-06j
  -9.72637013e-02-4.69019198e-11j ... -6.19990487e-07-2.59240522e-07j
   2.26403927e-08+2.14007358e-07j  3.96196754e-07-3.15022419e-07j]
 ...
 [ 7.38730421e-07+8.33450088e-07j  5.88204728e-07+1.28884835e-06j
  -6.19990487e-07+2.59240522e-07j ... -5.80081587e-06-3.42133522e-16j
   7.72764793e-07-3.88672983e-11j -7.68945426e-07-1.96464914e-10j]
 [ 1.06983116e-07+2.13041830e-07j -3.14634683e-08+9.98407927e-07j
   2.26403927e-08-2.14007358e-07j ...  7.72764793e-07+3.88672983e-11j
  -7.42289558e-06+2.64325991e-15j  1.30182946e-

In [67]:
# List of the frequencies available (in Hartree)
print(sus_ncfile.reader.wpoints)

[0.        +0.j         0.11111111+0.j         0.22222222+0.j
 0.33333333+0.j         0.44444444+0.j         0.55555556+0.j
 0.66666667+0.j         0.77777778+0.j         0.88888889+0.j
 1.        +0.j         0.        +0.15160798j 0.        +0.4155728j
 0.        +0.87516226j 0.        +1.67535397j]


## Reconstruction of $\chi^0(\boldsymbol{q}, \boldsymbol{G}, \boldsymbol{G'})$ from the symmetries with the important informations

If the file has already been analyzed, the matrix can be recovered in a .npy file (see list below) and its summary is available in the attached .txt file

- chi0GG_prim_eps21_FromSym_ngkpt444.npy

- chi0GG_prim_eps3_6_FromSym_ngkpt888.npy

In [738]:
#np.load(mat.npy)

If the matrix has not been build already, you can launch the following two boxes.

### Warning : This operation can take some time if symmetries have to be used, usually, for a reasonable size of input (about 100 MB), it takes a few minutes but time increases quickly up to a few hours for input file of size in the order of the GB. So make sure the job has not been launched already to avoid useless computations.

In [2]:
if nsym_var == 1:
    opt = "FullBZ"

else:
    opt = "FromSym" 
# choose the frequency you want (index of the frequency in the above table)
omega = 0

In [73]:
np.save("chi0GG_Conv_eps14_FromSym_ngkpt444_omega0_nband150", chi0GG)

In [5]:
chi0GG, ind_qG_to_vec, n1, n2, n3, ind_q_to_vec, ind_G_to_vec, G, nk = XGChi0.Build_Chi0GG(filename, opt, omega)

Basic Dict initialized
Dict with symmetry initialized
qGvec sorted
Dict of symmetry initialized


In [21]:
# Just check if n1 = n2 = n3 if dealing with a bulk material with three equivalent unit cell vectors

print(n1, n2, n3)

35 35 45


### Don't forget to save the matrix if you want it use again

Use a nice name in order to avoid confusion with other matrices (specify the type of unit cell (conv or prim), the ecuteps used (eps#), if the file has been built from sym (FromSym or FullBZ), the size of the kpoint mesh grid (ngkpt###) and the frequency)

### Information about the matrix

In [71]:
chi0GG_const = chi0GG[0, 0, 0]
sum_chi0GG = np.sum(chi0GG)
sum_norm_chi0GG = np.sum(np.abs(np.power(np.real(chi0GG), 2))),np.sum(np.abs(np.power(np.imag(chi0GG),2)))
#sum_abs_chi0GG = complex(np.sum(np.abs(np.real(chi0GG))),np.sum(np.abs(np.imag(chi0GG))))
max_r_chi0GG = np.max(np.abs(np.real(chi0GG)))
max_i_chi0GG = np.max(np.abs(np.imag(chi0GG)))

In [72]:
print("The constant component = ", chi0GG_const,"\nThe sum of all the components = ", sum_chi0GG,"\nThe sum of the norm of all components = ", sum_norm_chi0GG, "\nThe maximum absolute real value = ", max_r_chi0GG, "\nThe maximum absolute imaginary value = ", max_i_chi0GG,"\nThe shape of the matrix = ", chi0GG.shape)

The constant component =  (-5.972790262376293e-09+4.710217590852708e-26j) 
The sum of all the components =  (-2.6983802495612244+2.571870411049164e-09j) 
The sum of the norm of all components =  (23.74262418424471, 0.0023243191399856407) 
The maximum absolute real value =  0.10738244652748108 
The maximum absolute imaginary value =  0.0029017881024628878 
The shape of the matrix =  (64, 1141, 1141)


### Test for $\chi^0(\boldsymbol{q}, \boldsymbol{G}, \boldsymbol{G'})$ in order to check if it makes sens, at least at first sight

#### Test with respect to the reconstruction 
 

In [27]:
bool_GG = True

The number of kpoints should be equal to the size of ngkpt

In [28]:
if len(chi0GG[:,0, 0]) != np.product(ngkpt):
    print("This situation (the first dimension of the matrix being different from the size of the grid of kpoint concidered) should not happen, there might be a flaw in the code... Check if you reinitialized all the variables... If the problem persist, please let the developpers know") 
    bool_GG = False

The number of vector G should not have changed 

In [29]:
if len(chi0GG[0, :, 0]) != ng:
    print("This situation (the second dimension of the matrix being different than the size of the G-Sphere) should not happen, there might be a flaw in the code... Check if you reinitialized all the variables... If the problem persist, please let the developpers know") 
    bool_GG = False
if len(chi0GG[0, 0, :]) != ng:
    print("This situation (the third dimension of the matrix being different than the size of the G-Sphere) should not happen, there might be a flaw in the code... Check if you reinitialized all the variables... If the problem persist, please let the developpers know") 
    bool_GG = False

The maximum values of the complete matrix might not exceed those of the matrix provided by abinit.

In [37]:
max_i = 0
max_r = 0

for i in range(nkpt):
    chi0 = sus_ncfile.reader.read_wggmat(kpoints[i])
    chi0_mat = chi0.wggmat[omega]
    max_i = max(np.max(np.abs(np.imag(chi0_mat))), max_i)
    max_r = max(np.max(np.abs(np.real(chi0_mat))), max_r)

if max_r_chi0GG > max_r:
    print("This situation should not happen, there might be a flaw in the code... Check if you reinitialized all the variables... If the problem persist, please let the developpers know") 
    bool_GG = False
if max_i_chi0GG > max_i:
    print("This situation should not happen, there might be a flaw in the code... Check if you reinitialized all the variables... If the problem persist, please let the developpers know") 
    bool_GG = False
print(max_i)

0.023530036211013794


Finally, let's check the total number of component that did not make the cut in the matrix

In [31]:
nq, ng, ng_0 = chi0GG.shape

In [32]:
count = 0
for i in range(nq):
    for j in range(ng):
        for k in range(ng):
            if chi0GG[i,j,k] != 0:
                count +=1


In [33]:
print(100*count/(nq*ng**2), "% of the values are non-zero, if this number does not seem high enough check the size of the G-sphere and wheter a lot of its components are close to the surface or not... If yes, it is normal that most of them did not make the cut in order to preserve the symmetry. Increase the ecuteps variable to conserve a higher amount of qG vectors. Also check if the computation did complete. Else a lot of values are initialized at 0. If the problem persists with a bigger sphere (less than 60/%), please contact the developper, there might be a flaw in the code")

86.27502987696906 % of the values are non-zero, if this number does not seem high enough check the size of the G-sphere and wheter a lot of its components are close to the surface or not... If yes, it is normal that most of them did not make the cut in order to preserve the symmetry. Increase the ecuteps variable to conserve a higher amount of qG vectors. If the problem persists with a bigger sphere (less than 60/%), please contact the developper, there might be a flaw in the code


Test of the excpected characteristic. This section is subdivided with respect to the frequency of interest. Two cases are considered for metals : $\omega=0$ and $\omega\neq 0$.

## $\omega = 0$
If the frequency is null or imaginary, $\chi^0(\boldsymbol{q}, \boldsymbol{G}, \boldsymbol{G'})$ should be real and symmetric.

Let's at least hope the imaginary part is small compared to the real one

In [34]:
if max_i_chi0GG > max_r_chi0GG/100:
    print("This situation (the matrix not being real for the null frequency) should not happen, there might be a flaw in the code... Check if you reinitialized all the variables... If the problem persist, please let the developpers know") 
    bool_GG = False

This situation (the matrix not being real for the null frequency) should not happen, there might be a flaw in the code... Check if you reinitialized all the variables... If the problem persist, please let the developpers know


The matrix should be symmetric. Here, only the real part is considered (it has been checked just above that the imaginary part is neglegible, also, as the matrix is hermitician by definition in ABINIT, the equality would never be met)

In [38]:
if np.real(chi0GG[i, j, k]) != np.real(chi0GG[i, k, j]):
    print("This situation (the matrix not being symetric) should not happen, there might be a flaw in the code... Check if you reinitialized all the variables... If the problem persist, please let the developpers know")
    print(chi0GG[i, j, k], "is not equal to ", chi0GG[i, k, j])
    bool_GG = False

## $\omega \neq 0$

For a metal, all matrices should be symmetric. Let's pick a random component and see if its symetric is the same

In [39]:
i, j, k = random.randint(0, nq-1), random.randint(0, ng-1), random.randint(0, ng-1)

In [40]:
if chi0GG[i, j, k] != chi0GG[i, k, j]:
    print("This situation (the matrix not being symetric) should not happen, there might be a flaw in the code... Check if you reinitialized all the variables... If the problem persist, please let the developpers know")
    print(chi0GG[i, j, k], "is not equal to ", chi0GG[i, k, j])
    bool_GG = False

This situation (the matrix not being symetric) should not happen, there might be a flaw in the code... Check if you reinitialized all the variables... If the problem persist, please let the developpers know
(2.1706038921820436e-07+8.213497153519711e-07j) is not equal to  (2.1706038921820436e-07-8.213497153519711e-07j)


TO DO :

ADD saving of information

## Fourier Transform of $\chi^0(\boldsymbol{q}, \boldsymbol{G}, \boldsymbol{G'})$ and first information about $\chi^0(\boldsymbol{r}, \boldsymbol{r'})$

As for its reciprocal space counter part, you can either select one of the matrix already saved or obtain a new one (and save it afterwards if it is useful)

- chi0rr_prim_eps21_FromSym_ngkpt444_omega0

In [7]:
np.save("chi0rr_prim_eps17_FromSym_ngkpt101010_omega0_nband150", chi0rr)

In [6]:
chi0rr = XGChi0.FFT_chi0_from_Mat(chi0GG, ind_qG_to_vec, n1, n2, n3, ind_q_to_vec, ind_G_to_vec, G, nk)

Starting first FFT
Starting second FFT


### Information about the matrix

In [75]:
chi0rr_origin = chi0rr[0, 0, 0, 0, 0, 0]
sum_chi0rr = np.sum(chi0rr)
sum_per_pert = sum_chi0rr/chi0rr.size
sum_norm_chi0rr = np.sum(np.abs(np.power(np.real(chi0rr), 2))),np.sum(np.abs(np.power(np.imag(chi0rr),2)))
#sum_abs_chi0GG = complex(np.sum(np.abs(np.real(chi0GG))),np.sum(np.abs(np.imag(chi0GG))))
max_r_chi0rr = np.max(np.abs(np.real(chi0rr)))
max_i_chi0rr = np.max(np.abs(np.imag(chi0rr)))

In [76]:
print("The origin component = ", chi0rr_origin,"\nThe sum of all the components = ", sum_chi0rr, "\nThe average variation of the density for a given perturbation of 1 =", sum_per_pert, "\nThe sum of the norm of all components = ", sum_norm_chi0rr, "\nThe maximum absolute real value = ", max_r_chi0rr, "\nThe maximum absolute imaginary value = ", max_i_chi0rr,"\nThe shape of the matrix = ", chi0rr.shape)

The origin component =  (-2.698380249561217+2.5718704069698473e-09j) 
The sum of all the components =  (-3.174185629866031+5.808686864838819e-12j) 
The average variation of the density for a given perturbation of 1 = (-5.972790262448759e-09+1.0930069123080113e-20j) 
The sum of the norm of all components =  (12220515638.807056, 398523538.77993935) 
The maximum absolute real value =  588.5760689073397 
The maximum absolute imaginary value =  38.25490123453463 
The shape of the matrix =  (15, 15, 15, 54, 54, 54)


## Visulization tool in order to check if everything is correct

In [77]:
n1_go, n2_go, n3_go, n4_go, n5_go, n6_go = chi0rr.shape
# Check if the dimensions are consistant
print(chi0rr.shape)

(15, 15, 15, 54, 54, 54)


First check if the variations of the density along the three principal directions of the unit cells are the same for a potential change at the origin.

In [78]:
print(ngkpt)

[4, 4, 4]


In [79]:
X1 = np.linspace(0, ngkpt[0], n4_go,endpoint = False)
X2 = np.linspace(0, ngkpt[1], n5_go,endpoint = False)
X3 = np.linspace(0, ngkpt[2], n6_go,endpoint = False)

X1R = np.linspace(0, 1, n1_go, endpoint = False)
X2R = np.linspace(0, 1, n2_go, endpoint = False)
X3R = np.linspace(0, 1, n3_go, endpoint = False)

Y1 = np.real(chi0rr[0,0,0,:,0,0])
Y2 = np.real(chi0rr[0,0,0,0,:,0])
Y3 = np.real(chi0rr[0,0,0,0,0,:])

Y1R = np.real(chi0rr[:,0,0,0,0,0])
Y2R = np.real(chi0rr[0,:,0,0,0,0])
Y3R = np.real(chi0rr[0,0,:,0,0,0])

In [80]:
trace1 = go.Scatter(
                    x = X1,
                    y = Y1,
                    mode = "lines+markers",
                    name = "Axe0"
                    )

trace2 = go.Scatter(
                    x = X2,
                    y = Y2,
                    mode = "lines+markers",
                    name = "Axe1"
                    )

trace3 = go.Scatter(
                    x = X3,
                    y = Y3,
                    mode = "lines+markers",
                    name = "Axe2"
                    )

trace4 = go.Scatter(
                    x = X1R,
                    y = Y1R,
                    mode = "lines+markers",
                    name = "Axe0K"
                    )

trace5 = go.Scatter(
                    x = X2R,
                    y = Y2R,
                    mode = "lines+markers",
                    name = "Axe1K"
                    )

trace6 = go.Scatter(
                    x = X3R,
                    y = Y3R,
                    mode = "lines+markers",
                    name = "Axe2K"
                    )



data = [trace1, trace2, trace3, trace4, trace5, trace6]
layout = dict(title = '\chi^0(r, r\') with r_red ='+str([0, 0, 0])+' for different directions',
              xaxis= dict(title= 'Number of unit cells',ticklen= 5,zeroline= False),
             )
fig = go.Figure(dict(data = data, layout = layout))
fig.show()

Then, one can check if the function is symmetric for any situation : take a random vector in the unit cell and see if it is recovered when swapping both positions.

In [463]:
i, j, k, l, m, n = random.randint(0, n1_go-1), random.randint(0, n2_go-1), random.randint(0, n3_go-1), random.randint(0, n1_go-1), random.randint(0, n2_go-1), random.randint(0, n3_go-1)

## for $\omega=0$

In this case, you can check if the matrix is real before checking the symmetry. The use of the "FromSym" option leads to a non-negligible imaginary part reamining, yet, it is should be at least one order of magnitude less important than the real part. Check the max values above to see if it is respected

In [464]:
print(i, j, k, l, m, n)

1 2 1 1 1 0


In [465]:
if round(np.real(chi0rr[i, j, k, l, m, n]), 3) != round(np.real(chi0rr[l, m, n, i, j, k]), 3):
    print('The matrix is not symmetric, while the swapping of the two positions should not change the response, check wether the error is important and if yes, check the input file and the option used until this point. If everything checks out but the matrix still present big discrepancies, please notify the developpers')
    print(chi0rr[i, j, k, l, m, n],'is not equal to', chi0rr[l, m, n, i, j, k])


## for $\omega \neq 0$

In [466]:
if np.round(chi0rr[i, j, k, l, m, n]) ,3) != np.round(chi0rr[l, m, n, i, j, k]), 3):
    print('The matrix is not symmetric, while the swapping of the two positions should not change the response, check wether the error is important and if yes, check the input file and the option used until this point. If everything checks out but the matrix still present big discrepancies, please notify the developpers')
    print(chi0rr[i, j, k, l, m, n],'is not equal to', chi0rr[l, m, n, i, j, k])

SyntaxError: invalid syntax (<ipython-input-466-07763ca00d88>, line 1)

## If everything seems fine until here, you can start to analyze the matrix.

A 3D visualization tool is available hereunder. This gives an idea of the response but not its magnitude. Note that you can play with the number of points (N variable in the function) and the lower limit of the amplitude of the response you want to see (isomin variable in the function).DS_Store

A 2D plot is also availble with the possibility to plot the response along different direction and different distance from the perturbation

A place for convergence study is also available

In [46]:
R = [4, 2, 50]
# remember that R must be in the first unit cell

In [48]:
XGChi0.Vis_tool(chi0rr, R, A, B, C, ngkpt, natom, xred, N=10000, isomin=0)

In [49]:
X1_2D = np.linspace(0, ngkpt[0], n4_go,endpoint = False)
X2_2D = np.linspace(0, ngkpt[1], n5_go,endpoint = False)
X3_2D = np.linspace(0, ngkpt[2], n6_go,endpoint = False)

In [53]:
print(X3_2D)

[0.         0.00925926 0.01851852 0.02777778 0.03703704 0.0462963
 0.05555556 0.06481481 0.07407407 0.08333333 0.09259259 0.10185185
 0.11111111 0.12037037 0.12962963 0.13888889 0.14814815 0.15740741
 0.16666667 0.17592593 0.18518519 0.19444444 0.2037037  0.21296296
 0.22222222 0.23148148 0.24074074 0.25       0.25925926 0.26851852
 0.27777778 0.28703704 0.2962963  0.30555556 0.31481481 0.32407407
 0.33333333 0.34259259 0.35185185 0.36111111 0.37037037 0.37962963
 0.38888889 0.39814815 0.40740741 0.41666667 0.42592593 0.43518519
 0.44444444 0.4537037  0.46296296 0.47222222 0.48148148 0.49074074
 0.5        0.50925926 0.51851852 0.52777778 0.53703704 0.5462963
 0.55555556 0.56481481 0.57407407 0.58333333 0.59259259 0.60185185
 0.61111111 0.62037037 0.62962963 0.63888889 0.64814815 0.65740741
 0.66666667 0.67592593 0.68518519 0.69444444 0.7037037  0.71296296
 0.72222222 0.73148148 0.74074074 0.75       0.75925926 0.76851852
 0.77777778 0.78703704 0.7962963  0.80555556 0.81481481 0.824074

In [50]:
R_2D = [3, 3, 6]

In [54]:
Y1_2D = np.real(chi0rr[R_2D[0], R_2D[1], R_2D[2], :, R_2D[1], R_2D[2]])
Y2_2D = np.real(chi0rr[R_2D[0], R_2D[1], R_2D[2], R_2D[0], :, R_2D[2]])
Y3_2D = np.real(chi0rr[R_2D[0], R_2D[1], R_2D[2], R_2D[0], R_2D[1], :])



Y4_2D = np.real(chi0rr[R_2D[0], R_2D[1], R_2D[2], :, 0, 0])
Y5_2D = np.real(chi0rr[R_2D[0], R_2D[1], R_2D[2], 0, :, 0])
Y6_2D = np.real(chi0rr[R_2D[0], R_2D[1], R_2D[2], 0, 0, :])

Y1_2DK = np.real(chi0rr[:, R_2D[1], R_2D[2], R_2D[0], R_2D[1], R_2D[2]])
Y2_2DK = np.real(chi0rr[R_2D[0], :, R_2D[2], R_2D[0], R_2D[1], R_2D[2]])
Y3_2DK = np.real(chi0rr[R_2D[0], R_2D[1], :, R_2D[0], R_2D[1], R_2D[2]])

In [55]:
trace1 = go.Scatter(
                    x = X1_2D,
                    y = Y1_2D,
                    mode = "lines+markers",
                    name = "Axe0"
                    )

trace2 = go.Scatter(
                    x = X2_2D,
                    y = Y2_2D,
                    mode = "lines+markers",
                    name = "Axe1"
                    )

trace3 = go.Scatter(
                    x = X3_2D,
                    y = Y3_2D,
                    mode = "lines+markers",
                    name = "Axe2"
                    )

trace4 = go.Scatter(
                    x = X1_2D,
                    y = Y4_2D,
                    mode = "lines+markers",
                    name = "Axe0_origin"
                    )

trace5 = go.Scatter(
                    x = X2_2D,
                    y = Y5_2D,
                    mode = "lines+markers",
                    name = "Axe1_origin"
                    )

trace6 = go.Scatter(
                    x = X3_2D,
                    y = Y6_2D,
                    mode = "lines+markers",
                    name = "Axe2_origin"
                    )

trace7 = go.Scatter(
                    x = X1R,
                    y = Y1_2DK,
                    mode = "lines+markers",
                    name = "Axe0K"
                    )

trace8 = go.Scatter(
                    x = X2R,
                    y = Y2_2DK,
                    mode = "lines+markers",
                    name = "Axe1K"
                    )

trace9 = go.Scatter(
                    x = X3R,
                    y = Y3_2DK,
                    mode = "lines+markers",
                    name = "Axe2K"
                    )



data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9]
layout = dict(title = 'chi0(r, r\') with R ='+str(R_2D)+' for different directions',
              xaxis= dict(title= 'Number of unit cells',ticklen= 5,zeroline= False),
             )
fig = go.Figure(dict(data = data, layout = layout))
fig.show()

In [129]:
Y1_2D.shape

(72,)

TO DO :

- ADD analysis of data with scaling factor

- ADD Visual tests to observe convergence/accuracy and ways to conclude if the file is ok or not


In [132]:
print(sum_test)

(475.8924956197964+159.9808675017239j)
